In [ ]:
import pandas as pd
import numpy as np

import time
from scipy.linalg import hankel, eigh

from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('data/BatteryTemperature.csv', header=None, nrows=20000)

In [ ]:
df.head()

In [ ]:
with open('data/BatteryTemperature.csv', 'r') as file:
    for i in range(5):
        time.sleep(1)
        line = file.readline()
        print(line.split(',')[-1])

In [ ]:
# Draw Plot
def plot_df(data, title="Telemetry Plot", xlabel='Time', ylabel='Battery Temperature', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(data, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(data=df[1][:20000], title='TimeSeries of Train Data') 

In [ ]:
# Splitting into different sets

In [ ]:
X_train_data = df[1][:20000] # For training 
X_theta = df[1][20000:80000] # For threshold calculation
X_test = df[1][80000:] # For detection

In [ ]:
N = len(X_train_data)
L = N // 2
X_train = hankel(X_train_data[:L],X_train_data[L-1:]) # Creating trajectory matrix

In [ ]:
eigenValues, eigenVectors = eigh(np.matmul(X_train, X_train.T))
idx = eigenValues.argsort()[::-1]
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]

In [ ]:
# Sree plot
fig, ax = plt.subplots(1, 1, figsize=(10,8))
ax.plot(eigenValues[:50])
ax.set_title("Screeplot")
ax.set_ylabel("Eigen value ->")
ax.set_xlabel("Cardinal Number of Eigen value ->")

In [ ]:
# From the above scree plot, it is seen that the first eigen value is more 
# significant and the other eigen values are close to zero. Thus we choose r as 1
# and then we compute the U for projection and centroid in the signal subspace
r = 2
# Extracted Training signals
U, Sigma, V = np.linalg.svd(X_train)
V = V.T
# d = np.linalg.matrix_rank(X_train)
X_elem = np.array( [Sigma[i] * np.outer(U[:,i], V[:,i]) for i in range(0,r)] )
X_train_extracted = X_elem.sum(axis=0)
X_train_extracted_data = np.asarray(list(X_train_extracted[:,0]) + list(X_train_extracted[:,-1]))

U = eigenVectors[:,:r] # r as statistical dimension
UT = U.T
pX = np.matmul(UT,X_train_extracted)
centroid = np.mean(pX, axis=1)
centroid = centroid[:,np.newaxis]